In [1]:
import torch.nn as nn 
import torch 
from copy import deepcopy
class Policy(nn.Module):
    def __init__(self):
        super().__init__()
        self.core = nn.Sequential(
            nn.Linear(4,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,2)
        )
        
    def forward(self,x):
        return self.core(x)


class Value(nn.Module):
    def __init__(self):
        super().__init__()
        self.core = nn.Sequential(
            nn.Linear(4,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,1)
        )
        
    def forward(self,x):
        return self.core(x)


policyModel = Policy()

oldPolicyModel = deepcopy(policyModel)
def update_old_policy():
    global oldPolicyModel
    oldPolicyModel = deepcopy(policyModel)
    for param in oldPolicyModel.parameters():
        param.requires_grad = False

valueModel = Value()


KeyboardInterrupt: 

In [2]:
import gymnasium as gym 



env = gym.make("CartPole-v1")

In [3]:
def is_terminal(observation):
    x,v,theta , omega = observation 
    x_terminal = x <= -4.8 or x>=4.8
    theta_terminal = theta <= -0.209*7.5 or theta >= 0.209*7.5
    return x_terminal or theta_terminal


episodes = 10000

value_optimizer = torch.optim.Adam(valueModel.parameters(), lr=1e-3)
policy_optimizer = torch.optim.Adam(policyModel.parameters(),lr=1e-3)
valueLoss = torch.nn.functional.mse_loss
gamma = 0.99
epsilon =0.3
update_old_step = 10
update_old_policy()
for episode in range(episodes):
    done = False
    obs, info = env.reset()
    obs = torch.tensor(obs)
    observations = []
    rewards = []
    actions = []
    confidences= []
    
    with torch.no_grad():
        while not done:
            
            observations.append(obs)
            logits = policyModel(obs)
            probs = torch.softmax(logits,dim=-1)

            action = torch.multinomial(probs, num_samples=1).item()
            confidences.append(probs[action]) 
            actions.append(action)
            obs, reward, terminated, truncated , info = env.step(action)
            rewards.append(reward)
            obs = torch.tensor(obs)
            done = is_terminal(obs)
        
        
        
        discontinued_rewards = []
        reward = 0 
        observation_tensor = torch.stack(observations) 
        for i in range(-1,-len(rewards) -1,-1):
            reward = rewards[i] + gamma * reward
            discontinued_rewards.append(reward)


        discontinued_rewards.reverse()
        discontinued_rewards = torch.tensor(discontinued_rewards,dtype=torch.float32) 
  
    
    reward_preds = valueModel(observation_tensor)   
    loss = valueLoss(reward_preds,discontinued_rewards)
    value_optimizer.zero_grad()
    loss.backward()
    value_optimizer.step()
    reward_preds = reward_preds.detach()

    total_loss = 0
    advantages = []
    for i in range(len(observations)):
        predicted_reward = reward_preds[i]
        reward = discontinued_rewards[i]
        advantage = reward - predicted_reward 
        advantages.append(advantage)
    advantages = torch.stack(advantages)
    advantages = (advantages - advantages.mean()) / (advantages.std(unbiased=False) + 1e-8)
    for i in range(len(observations)):
        advantage = advantages[i]
        current_logits = policyModel(observations[i])
        current_log_probs = torch.log_softmax(current_logits,dim=-1)

        current_confidence = current_log_probs[actions[i]]

        old_logits = oldPolicyModel(observations[i])
       
       
        old_log_probs = torch.log_softmax(old_logits,dim=-1)

        old_confidence =old_log_probs[actions[i]]
        
        confidence_ratio = torch.exp(current_confidence - old_confidence)
        loss = confidence_ratio* advantage 
        loss2 =torch.clip( (confidence_ratio) , 1-epsilon , 1+epsilon)*advantage 
        loss = -torch.min(loss, loss2)
        
        total_loss += loss 
    total_loss /= len(observations)
    print('episode: ',episode + 1, ' loss: ',total_loss)
    policy_optimizer.zero_grad()
    total_loss.backward()
    policy_optimizer.step()
    if  (episode + 1)  % update_old_step == 0:
        update_old_policy()
    







    

    

c:\Users\beca\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\classic_control\cartpole.py:214: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(
C:\Users\beca\AppData\Local\Temp\ipykernel_4872\718079957.py:56: UserWarning: Using a target size (torch.Size([34])) that is different to the input size (torch.Size([34, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = valueLoss(reward_preds,discontinued_rewards)
C:\Users\beca\AppData\Local\Temp\ipykernel_4872\718079957.py:56: UserWarning: Using a target size (torch.Size([43])) that is different to the input size (torch.Size([43, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = valueLoss(reward

episode:  1  loss:  tensor([-6.4864e-08], grad_fn=<DivBackward0>)
episode:  2  loss:  tensor([-0.0046], grad_fn=<DivBackward0>)
episode:  3  loss:  tensor([0.0160], grad_fn=<DivBackward0>)
episode:  4  loss:  tensor([-0.0064], grad_fn=<DivBackward0>)
episode:  5  loss:  tensor([0.0114], grad_fn=<DivBackward0>)


C:\Users\beca\AppData\Local\Temp\ipykernel_4872\718079957.py:56: UserWarning: Using a target size (torch.Size([36])) that is different to the input size (torch.Size([36, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = valueLoss(reward_preds,discontinued_rewards)
C:\Users\beca\AppData\Local\Temp\ipykernel_4872\718079957.py:56: UserWarning: Using a target size (torch.Size([37])) that is different to the input size (torch.Size([37, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = valueLoss(reward_preds,discontinued_rewards)
C:\Users\beca\AppData\Local\Temp\ipykernel_4872\718079957.py:56: UserWarning: Using a target size (torch.Size([70])) that is different to the input size (torch.Size([70, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = valueLoss(reward_preds,discontinued_rewards)
C

episode:  6  loss:  tensor([0.0135], grad_fn=<DivBackward0>)
episode:  7  loss:  tensor([0.0155], grad_fn=<DivBackward0>)


C:\Users\beca\AppData\Local\Temp\ipykernel_4872\718079957.py:56: UserWarning: Using a target size (torch.Size([38])) that is different to the input size (torch.Size([38, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = valueLoss(reward_preds,discontinued_rewards)


...

C:\Users\beca\AppData\Local\Temp\ipykernel_4872\718079957.py:56: UserWarning: Using a target size (torch.Size([23])) that is different to the input size (torch.Size([23, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = valueLoss(reward_preds,discontinued_rewards)


episode:  722  loss:  tensor([0.0003], grad_fn=<DivBackward0>)
episode:  723  loss:  tensor([0.0012], grad_fn=<DivBackward0>)
episode:  724  loss:  tensor([-2.1790e-05], grad_fn=<DivBackward0>)
episode:  725  loss:  tensor([-0.0134], grad_fn=<DivBackward0>)
episode:  726  loss:  tensor([0.0017], grad_fn=<DivBackward0>)
episode:  727  loss:  tensor([0.0108], grad_fn=<DivBackward0>)
episode:  728  loss:  tensor([0.0093], grad_fn=<DivBackward0>)
episode:  729  loss:  tensor([0.0049], grad_fn=<DivBackward0>)
episode:  730  loss:  tensor([0.0024], grad_fn=<DivBackward0>)
episode:  731  loss:  tensor([-1.7881e-08], grad_fn=<DivBackward0>)
episode:  732  loss:  tensor([-0.0015], grad_fn=<DivBackward0>)
episode:  733  loss:  tensor([0.0005], grad_fn=<DivBackward0>)
episode:  734  loss:  tensor([-0.0015], grad_fn=<DivBackward0>)
episode:  735  loss:  tensor([-0.0057], grad_fn=<DivBackward0>)
episode:  736  loss:  tensor([-0.0016], grad_fn=<DivBackward0>)
episode:  737  loss:  tensor([-0.0059], 

C:\Users\beca\AppData\Local\Temp\ipykernel_4872\718079957.py:56: UserWarning: Using a target size (torch.Size([22])) that is different to the input size (torch.Size([22, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = valueLoss(reward_preds,discontinued_rewards)


episode:  1825  loss:  tensor([0.0750], grad_fn=<DivBackward0>)
episode:  1826  loss:  tensor([0.0677], grad_fn=<DivBackward0>)
episode:  1827  loss:  tensor([0.0604], grad_fn=<DivBackward0>)
episode:  1828  loss:  tensor([0.0388], grad_fn=<DivBackward0>)
episode:  1829  loss:  tensor([0.0135], grad_fn=<DivBackward0>)
episode:  1830  loss:  tensor([0.0017], grad_fn=<DivBackward0>)
episode:  1831  loss:  tensor([8.5520e-08], grad_fn=<DivBackward0>)
episode:  1832  loss:  tensor([-0.0062], grad_fn=<DivBackward0>)
episode:  1833  loss:  tensor([0.3133], grad_fn=<DivBackward0>)
episode:  1834  loss:  tensor([0.0012], grad_fn=<DivBackward0>)
episode:  1835  loss:  tensor([0.0471], grad_fn=<DivBackward0>)
episode:  1836  loss:  tensor([0.0867], grad_fn=<DivBackward0>)
episode:  1837  loss:  tensor([0.1300], grad_fn=<DivBackward0>)
episode:  1838  loss:  tensor([0.1603], grad_fn=<DivBackward0>)
episode:  1839  loss:  tensor([0.2141], grad_fn=<DivBackward0>)
episode:  1840  loss:  tensor([0.25

KeyboardInterrupt: 

In [5]:
import gymnasium as gym 



env = gym.make("CartPole-v1",render_mode='human')

In [6]:


for episode in range(episodes):
    done = False
    obs, info = env.reset(seed=42)
    obs = torch.tensor(obs)
    observations = []
    rewards = []
    actions = []
    confidences= []
    with torch.no_grad():
        while not done:
            
            observations.append(obs)
            logits = policyModel(obs)
            probs = torch.softmax(logits,dim=-1)

            action = torch.multinomial(probs, num_samples=1).item()
            confidences.append(probs[action])
            actions.append(action)
            obs, reward, terminated, truncated , info = env.step(action)
            rewards.append(reward)
            obs = torch.tensor(obs)
            done = is_terminal(obs)
        print('episode: ',episode + 1)
        

    

    

    

c:\Users\beca\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\classic_control\cartpole.py:214: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


episode:  1
episode:  2
episode:  3
episode:  4
episode:  5
episode:  6
episode:  7
episode:  8
episode:  9
episode:  10
episode:  11
episode:  12
episode:  13
episode:  14
episode:  15
episode:  16
episode:  17
episode:  18
episode:  19
episode:  20


KeyboardInterrupt: 